# Realizando Web Scraping pelas patentes de Belo Horizonte registradas no USPTO utilizando BeautifulSoup

## Importação de Bibliotecas

In [1]:
from bs4 import BeautifulSoup 
import urllib.request as urllib_request
import pandas as pd
import csv
import re
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError

## Lendo o HTML da primeira patente da lista

In [3]:
from urllib.request import urlopen

url = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=2&f=G&l=50&co1=AND&d=PTXT&s1="Belo+Horizonte".INCI.&OS=IC/"Belo+Horizonte"'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0'}
try:
    req = Request(url, headers=headers)
    response = urlopen(req)
except HTTPError as e:
    print(e.status, e.reason)
except URLError as e:
    print(e.status)
html = response.read()


## Tratando o HTML

In [4]:
html = html.decode('utf-8')

In [5]:
def trata_html(input):
    return " ".join(input.split()).replace('> <', '><')

In [6]:
html = trata_html(html)

In [7]:
soup = BeautifulSoup(html, 'html5lib')

## Obtendo dados da patente

In [90]:
#Dicionário que armazenará os dados da patente
patente = {}

In [146]:
print('( Certificate of Correction )' in html)

True


#### Trechos de código como o acima serão utilizados para verificar se uma determinada informação está presente na página da patente, já que em algumas patentes não possuem Assignee, por exemplo.

In [19]:
for i in range(259):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258


In [18]:
soups = soup.findAll('td', {'align':'right'})[2].find('b')

In [14]:
for table in tabelas:
    if 'Current International Class' in table.getText():
        tabela_classes = table.findAll('tr')[-1].findAll('td')[1]

In [39]:
for tr in soup.findAll('tr'):
    if 'Current International Class' in tr.getText():
        print(tr.findAll('td')[1].getText())

C01G 23/047 (20060101); C01G 23/00 (20060101); C22B 34/12 (20060101); C22B 34/00 (20060101); C01G 023/00 (); C01G 023/04 ()


## Iterando por todas as patentes disponíveis e obtendo seus dados

In [40]:
url = 'http://patft.uspto.gov/netacgi/nph-Parser?Sect1=PTO2&Sect2=HITOFF&p=1&u=%2Fnetahtml%2FPTO%2Fsearch-bool.html&r=1&f=G&l=50&co1=AND&d=PTXT&s1=%22Belo+Horizonte%22.INCI.&OS=IC/%22Belo+Horizonte%22&RS=IC/%22Belo+Horizonte%22'

req = Request(url, headers=headers)
response = urlopen(req)
html = response.read()
html = html.decode('utf-8')
html = trata_html(html)
soup = BeautifulSoup(html, 'html5lib')
print(soup.findAll('font', {'size': '+1'})[0].getText())
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:73.0) Gecko/20100101 Firefox/73.0'}
patentes = []
patente = {}
patente['Título'] = soup.findAll('font', {'size': '+1'})[0].getText()
patente['Número'] = soup.find('title').getText()[22:]
patente['Data de Publicação'] = soup.findAll('td', {'align':'right'})[2].find('b').getText()
for tr in soup.findAll('tr'):
    if 'Current International Class' in tr.getText():
        patente['Classes Tecnológicas'] = tr.findAll('td')[1].getText()
if('( Certificate of Correction )' in html):
    patente['Inventores'] = soup.findAll('table')[4].findAll('tr')[0].getText()[10:]
    temp_tables = soup.findAll('table')[4].findAll('tr')
    for table in temp_tables:
        texto = table.getText()
        if 'Filed' in texto and 'PCT' not in texto:
            patente['Data de Submissão'] = table.find('td').getText()
else:
    patente['Inventores'] = soup.findAll('table')[3].findAll('tr')[0].getText()[10:]
    temp_tables = soup.findAll('table')[3].findAll('tr')
    for table in temp_tables:
        texto = table.getText()
        if 'Filed' in texto and 'PCT' not in texto:
            patente['Data de Submissão'] = table.find('td').getText()
if('Assignee' in html):
    if('Applicant:' in html):
        patente['Assignee']=soup.findAll('td', align="left", width="90%")[2].getText()
    else:
        patente['Assignee']=soup.findAll('td', align="left", width="90%")[1].getText()
else:
    patente['Assignee']='Self-Assigneed'
patentes.append(patente)
next = soup.find('table').findAll('tr')[1].findAll('a')[2].get('href')
#print(next)
for i in range(262):
    patente = {}
    #print(i+2)
    url = 'http://patft.uspto.gov'+next
    req = Request(url, headers=headers)
    response = urlopen(req)
    html = response.read()
    html = html.decode('utf-8')
    html = trata_html(html)
    soup = BeautifulSoup(html, 'html5lib')
    print(soup.findAll('font', {'size': '+1'})[0].getText())
    patente['Título'] = soup.findAll('font', {'size': '+1'})[0].getText()
    patente['Número'] = soup.find('title').getText()[22:]
    patente['Data de Publicação'] = soup.findAll('td', {'align':'right'})[2].find('b').getText()
    for tr in soup.findAll('tr'):
        if 'Current International Class' in tr.getText():
            patente['Classes Tecnológicas'] = tr.findAll('td')[1].getText()

    if('( Certificate of Correction )' in html):
        patente['Inventores'] = soup.findAll('table')[4].findAll('tr')[0].getText()[10:]
        temp_tables = soup.findAll('table')[4].findAll('tr')
        for table in temp_tables:
            texto = table.getText()
            if 'Filed' in texto and 'PCT' not in texto:
                patente['Data de Submissão'] = table.find('td').getText()
    else:
        patente['Inventores'] = soup.findAll('table')[3].findAll('tr')[0].getText()[10:]
        temp_tables = soup.findAll('table')[3].findAll('tr')
        for table in temp_tables:
            texto = table.getText()
            if 'Filed' in texto and 'PCT' not in texto:
                patente['Data de Submissão'] = table.find('td').getText()
    if('Assignee' in html):
        if('Applicant:' in html):
            patente['Assignee']=soup.findAll('td', align="left", width="90%")[2].getText()
        else:
            patente['Assignee']=soup.findAll('td', align="left", width="90%")[1].getText()
    else:
        patente['Assignee']='Self-Assigneed'
    patentes.append(patente)
    # Lidando com o link 'next' em diferentes partes da pagina
    if (i+2)%50 == 0:
        if (i+2) == 50:
            temp_url = 'http://patft.uspto.gov'+soup.find('table').findAll('tr')[1].findAll('a')[1].get('href')
            temp_req = Request(temp_url, headers=headers)
            response = urlopen(temp_req)
            temp_html = response.read()
            temp_html = temp_html.decode('utf-8')
            temp_html = trata_html(temp_html)
            temp_soup = BeautifulSoup(temp_html, 'html5lib')
            next = temp_soup.findAll('table')[1].find('a').get('href')
        else:
            temp_url = 'http://patft.uspto.gov'+soup.find('table').findAll('tr')[1].findAll('a')[2].get('href')
            temp_req = Request(temp_url, headers=headers)
            response = urlopen(temp_req)
            temp_html = response.read()
            temp_html = temp_html.decode('utf-8')
            temp_html = trata_html(temp_html)
            temp_soup = BeautifulSoup(temp_html, 'html5lib')
            next = temp_soup.findAll('table')[1].find('a').get('href')
    elif ((((i+2)%50)>1) and ((i+2)>51)) and ((i+2)<250):
        next = soup.find('table').findAll('tr')[1].findAll('a')[4].get('href')
        #print(next)
        #print('caso 2')
    elif (((i+2)<50 or (i+2)%50==1) and ((i+2)<251)) or ((i+2)>251):
        next = soup.find('table').findAll('tr')[1].findAll('a')[3].get('href')
    elif (i+2)==251:
        next = soup.find('table').findAll('tr')[1].findAll('a')[2].get('href')
        #print('caso 3')
dataset = pd.DataFrame(patentes)
dataset.to_csv('./output/belo_horizonte_nova.csv', sep = ';', index = False, encoding = 'utf-8-sig')
    

Graphene supercapacitor design and manufacture 
Automatic system and method for measuring and machining the end of tubular elements 
System, method, and computer program product for over-constraint/deadcode detection in a formal verification 
Residential automation system, equipment and process that is easy to install, configure and use 
Adjusted mutable aspect content selection 
Method of managing a set of lids, stacked and inserted in a package, transport device for transporting a set of stacked lids and inserted in a package, and system for producing palletized set of lids 
Fabrication of carbon nanoribbons from carbon nanotube arrays 
System, method, and computer program product for displaying multiple traces while debugging during a formal verification 
Sparger device for sparging a fluid into a slurry tank, flotation apparatus and system having such sparger device, and method for setting control fluid pressure of a sparger device 
Techniques for controlling a portable welding sys

Rebroadcasting of advertisements in a social network 
Portable welding system 
Selection of a secondary component carrier based on interference information 
Synthetic PnTx(19) peptide, pharmaceutical compositions and use 
Following content providers in a social context 
Providing resources to users in a social network system 
Automatic classification of the degree of maturation for iron ore pellets 
Handle design for a portable welding system 
Flexible lighting devices 
Ore fine agglomerate to be used in sintering process and production process of ore fines agglomerate 
Method for producing an agglomerate made of fine material containing metal oxide for use as a blast furnace feed material 
Recovery of base metals from sulphide ores and concentrates 
Process for the improvement of reducibility of ore pellets 
Side plate design for a portable welding system 
System and method for sharing of athletic performance data 
Mouse model of cholinergic dysfunction to evaluate cognitive enhancers

 Method and apparatus for facilitating removal of a corneal graft 
 Harvested tissue heart valve with sewing rim 
 Bioactive composites comprising silane functionalized polyaryl polymers 
 Silane functionalized polyaromatic polymers and methods for their production 
 Process for the production of titanium concentrate from anatase ores with high utilization of the iron contents of the ore 
 Recombinant human beta-CIS interferon 
 Advertising device for beverages with bubbling luminous animation 
 Single suture biological tissue aortic stentless valve 
 System for the injection of gasses into containers containing metals or fused alloys 
 Prostaglandin conjugates for treating or preventing bone disease 
 Total mitral heterologous bioprosthesis to be used in mitral or tricuspid heart replacement 
 Thrombolytic agents with antithrombotic activity 
 Portable basketball return device 
 Device and method for implanting an intralamellar ring in the correction of ametropias 
 Vaccine against he

### Caso você queira testar o Scraper, rode a célula acima e a primeira. Além disso, é necessário que haja uma pasta chamada "output" no mesmo diretório do Jupyter Notebook

In [33]:
patentes

[{'Título': 'Graphene supercapacitor design and manufacture ',
  'Número': '10714273',
  'Data de Publicação': ' July 14, 2020 ',
  'Classes Tecnológicas': 'H01G 11/36\xa0(20130101); H01G 11/56\xa0(20130101); H01G 11/32\xa0(20130101); H01G 11/10\xa0(20130101); H01G 11/86\xa0(20130101); H01G 11/58\xa0(20130101)',
  'Inventores': 'Dayrell; Ivan Araujo (Belo HorizonteBR) ',
  'Data de Submissão': 'June 21, 2019',
  'Assignee': 'Self-Assigneed'},
 {'Título': 'Automatic system and method for measuring and machining the end of tubular elements ',
  'Número': '10710163',
  'Data de Publicação': ' July 14, 2020 ',
  'Classes Tecnológicas': 'B23B 5/08\xa0(20060101); G01B 11/08\xa0(20060101); G01N 21/952\xa0(20060101); G01N 21/954\xa0(20060101); G01B 11/12\xa0(20060101); B23Q 17/24\xa0(20060101); B23Q 17/20\xa0(20060101)',
  'Inventores': 'Silveira E Silva; Julio Marcio (Belo HorizonteBR), Jardim De Souza; Marcelo (Belo HorizonteBR), Leal Carvalho; Felipe (Belo HorizonteBR), Alves Pinheiro; Gust

## Desenvolvimentos futuros

### Ainda são necessários implementar rotinas para obter as outras informações das patentes, como assignee, etc.

### Além disso, também é necessário automatizar determinados processos, como alguns número utilizados nos condicionais acima, os quais podem ser obtidos por meio da página inicial